In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
import os
import zipfile
import requests

# GitHub URL for dataset turned into zip
zip_url = "https://github.com/chascc/DL-Model-Training/archive/refs/heads/main.zip"
local_zip = "data.zip"

# Download and extract dataset
response = requests.get(zip_url)
with open(local_zip, "wb") as f:
    f.write(response.content)

with zipfile.ZipFile(local_zip, "r") as zip_ref:
    zip_ref.extractall(".")
    
# Updates the base directory for dataset
base_url = "./DL-Model-Training-main/data/natural_images"
print("Dataset:", base_url)

In [ ]:
# Image dimensions
img_h = 128
img_w = 128

# Data generators with validation split that normalizes pixel values to range (0, 1)
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training data loader
load_train = data_gen.flow_from_directory(
    base_url,
    # Resizes images to 128x128
    target_size=(img_h, img_w),
    batch_size=32,
    # Multi class classification
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Validation data loader
load_val = data_gen.flow_from_directory(
    base_url,
    target_size=(img_h, img_w),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)



In [ ]:
# Displays the class labels
class_name = list(load_train.class_indices.keys())
print("Class labels:", class_name)

# Displays example images
plt.figure(figsize=(10, 5))
for images, labels in load_train:
    # Displays 6 randomized example images
    for i in range(6): 
        # Create a 3x3 grid for images to test for successful reference
        plt.subplot(2, 3, i + 1)
        plt.imshow(images[i])
        # Uses the class label as title
        plt.title(class_name[np.argmax(labels[i])])
        plt.axis('off')
    break
# Alters the spacing between grids
plt.tight_layout()
plt.show()
0

In [ ]:
# Defines the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_h, img_w, 3)), # Convolution 1
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'), # Convolution 2
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'), # Convolution 3
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'), # Full Connection
    Dropout(0.5), 
    # Output layer with softmax activation
    Dense(len(class_name), activation='softmax')
])

# Displays a model summary
model.summary()


In [12]:
# Compiles the CNN model using adam optimizer and loss function for multi class
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Trains the CNN model using training data and val data with 10 epochs
history = model.fit(load_train, validation_data=load_val, epochs=10)
# Saves for repeated use
model.save('natural_images_cnn_model.keras')


In [ ]:
# Evaluates the model
loss, accuracy = model.evaluate(load_val)
print(f"Total Loss of Validation: {loss:.4f}")
print(f"Total Accuracy of Validation: {accuracy:.4f}")

In [ ]:
# Plots the training history
plt.figure(figsize=(12, 5))

# Plots the training and validation accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plots the training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
# Predict some images from validation data
import random

# Gets a batch of images from the folder at random
images, labels = next(load_val)
sample_indices = random.sample(range(len(images)), 6)

plt.figure(figsize=(10, 5))
for i, idx in enumerate(sample_indices):
    plt.subplot(2, 3, i + 1)
    plt.imshow(images[idx])
    # Predicts the class for the image using newly trained model
    pred_class = class_name[np.argmax(model.predict(images[idx][np.newaxis, ...]))]
    # Displays the actual name for class
    true_class = class_name[np.argmax(labels[idx])]
    plt.title(f"True: {true_class}\nPred: {pred_class}")
    plt.axis('off')
plt.tight_layout()
plt.show()